In [2]:
import pandas as pd
import cv2
import numpy as np
import os
import re
import ast
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image

In [2]:
# so I would like to generate a binary mask for blood vessel and glomerulus separately.
# the combined bv and glo annotations and mask is already saved in obj_detect_bv_glo.xlsx
# generate bv_mask and glo.
annotations_df_src = r"\\fatherserverdw\Kevin\hubmap\obj_detect_bv_glo.xlsx"
annotations_df = pd.read_excel(annotations_df_src)

In [3]:
bv_ann_df = annotations_df[annotations_df["type"] == "blood_vessel"]
bv_ann_df

,Unnamed: 0,id,type,coordinates,mask,rle,source_wsi,i,j
0,0,0006ff2aa7cd,blood_vessel,"[[[283, 109], [282, 109], [281, 109], [280, 10...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,['139350 10 139860 17 140370 22 140880 27 1413...,2,16896,16420
1,1,0006ff2aa7cd,blood_vessel,"[[[104, 292], [103, 292], [102, 292], [101, 29...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,['37108 4 37618 8 38129 11 38640 14 39152 15 3...,2,16896,16420
2,2,0006ff2aa7cd,blood_vessel,"[[[505, 442], [504, 442], [503, 442], [502, 44...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,['250783 11 251292 20 251801 26 252311 30 2528...,2,16896,16420
3,3,0006ff2aa7cd,blood_vessel,"[[[375, 477], [374, 477], [373, 477], [372, 47...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,['167352 5 167863 7 168374 10 168886 11 169398...,2,16896,16420
4,4,0006ff2aa7cd,blood_vessel,"[[[368, 410], [367, 410], [366, 410], [365, 41...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,['171880 10 172389 18 172899 24 173410 28 1739...,2,16896,16420
...,...,...,...,...,...,...,...,...,...
16049,16049,ffd3d193c71e,blood_vessel,"[[[184, 308], [183, 308], [182, 308], [181, 30...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,['88875 8 89385 11 89896 13 90407 15 90918 16 ...,3,7680,16896
16050,16050,ffd3d193c71e,blood_vessel,"[[[42, 92], [41, 92], [40, 92], [39, 92], [38,...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,['14393 12 14904 20 15415 23 15925 28 16434 40...,3,7680,16896
16051,16051,ffd3d193c71e,blood_vessel,"[[[287, 480], [286, 480], [285, 480], [284, 48...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,['140243 6 140750 13 141261 15 141772 17 14228...,3,7680,16896
16052,16052,ffd3d193c71e,blood_vessel,"[[[493, 388], [492, 388], [491, 388], [490, 38...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,['244001 5 244512 7 244603 6 245023 9 245112 1...,3,7680,16896


In [9]:
generate_mask = False
if generate_mask:
    blank_img = np.zeros((512,512),dtype=np.uint8)
    grouped_df = bv_ann_df.groupby('id')
    bv_mask_save_src = r"\\fatherserverdw\Kevin\hubmap\unet++\masks\blood_vessel"
    for id_, group in tqdm(grouped_df):
        mask_img = blank_img.copy()
        for coords in group['coordinates']:
            ex_coords = np.array(ast.literal_eval(coords))
            cv2.fillPoly(mask_img, [ex_coords], 1)
        cv2.imwrite(os.path.join(bv_mask_save_src,"{}.png".format(id_)), mask_img)

100%|██████████| 1622/1622 [00:23<00:00, 68.85it/s] 


In [4]:
glo_ann_df = annotations_df[annotations_df["type"] == "glomerulus"]

glo_ann_df

,Unnamed: 0,id,type,coordinates,mask,rle,source_wsi,i,j
16054,16054,0006ff2aa7cd,glomerulus,"[[[167, 249], [166, 249], [165, 249], [164, 24...",[[0 1 1 ... 0 0 0]\n [1 1 1 ... 0 0 0]\n [1 1 ...,['2 105 513 107 1025 111 1537 113 2049 114 256...,2,16896,16420
16055,16055,00168d1b7522,glomerulus,"[[[511, 216], [511, 216], [510, 216], [509, 21...",[[0 0 0 ... 1 1 1]\n [0 0 0 ... 1 1 1]\n [0 0 ...,['199180 13 199682 39 200193 43 200705 51 2012...,2,14848,14884
16056,16056,004daf1cbe75,glomerulus,"[[[73, 232], [72, 232], [71, 232], [70, 232], ...",[[0 1 1 ... 0 0 0]\n [1 1 1 ... 0 0 0]\n [1 1 ...,['2 222 513 223 1025 223 1537 223 2049 223 256...,3,6144,11264
16057,16057,005715f0a313,glomerulus,"[[[511, 507], [511, 507], [511, 506], [511, 50...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,['208686 21 209177 51 209681 67 210191 72 2106...,3,8192,10752
16058,16058,005715f0a313,glomerulus,"[[[511, 507], [511, 507], [511, 506], [511, 50...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,['208686 21 209177 51 209681 67 210191 72 2106...,3,8192,10752
...,...,...,...,...,...,...,...,...,...
16616,16616,ff99cdef0f2a,glomerulus,"[[[511, 101], [511, 101], [511, 100], [510, 10...",[[0 0 0 ... 1 1 1]\n [0 0 0 ... 1 1 1]\n [0 0 ...,['240130 14 240641 23 241153 25 241665 30 2421...,4,5120,24064
16617,16617,ffd3d193c71e,glomerulus,"[[[450, 193], [449, 193], [448, 193], [447, 19...",[[0 0 0 ... 1 1 1]\n [0 0 0 ... 1 1 1]\n [0 0 ...,['157224 34 157729 45 158223 68 158731 74 1592...,3,7680,16896
16618,16618,ffd3d193c71e,glomerulus,"[[[450, 193], [449, 193], [448, 193], [447, 19...",[[0 0 0 ... 1 1 1]\n [0 0 0 ... 1 1 1]\n [0 0 ...,['157224 34 157729 45 158223 68 158731 74 1592...,3,7680,16896
16619,16619,ffd3d193c71e,glomerulus,"[[[199, 511], [198, 511], [197, 511], [196, 51...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,['13259 8 13761 23 14269 30 14774 40 15282 45 ...,3,7680,16896


In [8]:
generate_mask = False
if generate_mask:
    blank_img = np.zeros((512,512),dtype=np.uint8)
    grouped_df = glo_ann_df.groupby('id')
    glo_mask_save_src = r"\\fatherserverdw\Kevin\hubmap\unet++\masks\glomerulus"
    for id_, group in tqdm(grouped_df):
        mask_img = blank_img.copy()
        for coords in group['coordinates']:
            ex_coords = np.array(ast.literal_eval(coords))
            cv2.fillPoly(mask_img, [ex_coords], 1)
        cv2.imwrite(os.path.join(glo_mask_save_src,"{}.png".format(id_)), mask_img)

100%|██████████| 435/435 [00:05<00:00, 76.12it/s] 


In [7]:
# create blood vessel and glomerulus dataframes separately. Should include paths to images and masks
bv_train_df = pd.DataFrame(columns=["id","image_path","mask_path"])
bv_img_list = [os.path.join(r"\\fatherserverdw\Kevin\hubmap\unet++\images",x) for x in os.listdir(r"\\fatherserverdw\Kevin\hubmap\unet++\images") if x.endswith('.tif')]
bv_mask_list = [os.path.join(r"\\fatherserverdw\Kevin\hubmap\unet++\masks\blood_vessel",x) for x in os.listdir(r"\\fatherserverdw\Kevin\hubmap\unet++\images") if x.endswith('.tif')]
bv_mask_list = [x.replace(".tif",".png") for x in bv_mask_list]
bv_ids = [os.path.basename(x).split(".tif")[0] for x in bv_img_list]
bv_train_df["id"] = bv_ids
bv_train_df["image_path"] = bv_img_list
bv_train_df["mask_path"] = bv_mask_list
match_df = bv_ann_df.groupby('id').agg({'source_wsi': 'first'}).reset_index()
bv_train_df = bv_train_df.merge(match_df[["id","source_wsi"]],on='id')
bv_train_df
bv_train_df.to_excel(r'\\fatherserverdw\Kevin\hubmap\unet++\bv_train_df.xlsx')

In [90]:
glo_train_df = pd.DataFrame(columns=["id","image_path","mask_path"])
glo_mask_list = [os.path.join(r"\\fatherserverdw\Kevin\hubmap\unet++\masks\glomerulus",x) for x in os.listdir(r"\\fatherserverdw\Kevin\hubmap\unet++\masks\glomerulus") if x.endswith(".png")]
glo_ids = [os.path.basename(x).split(".png")[0] for x in glo_mask_list]
glo_img_list = [os.path.join(r"\\fatherserverdw\Kevin\hubmap\unet++\images",x) + ".tif" for x in glo_ids]
glo_train_df["id"] = glo_ids
glo_train_df["image_path"] = glo_img_list
glo_train_df["mask_path"] = glo_mask_list
match_df = glo_ann_df.groupby('id').agg({'source_wsi': 'first'}).reset_index()
glo_train_df = glo_train_df.merge(match_df[["id","source_wsi"]],on='id')
glo_train_df
glo_train_df.to_excel(r'\\fatherserverdw\Kevin\hubmap\unet++\glo_train_df.xlsx')

### Try adding dilation to the masks:

In [10]:
from skimage.morphology import binary_dilation
saved_mask_src = r"\\fatherserverdw\Kevin\hubmap\unet++\masks\blood_vessel"
saved_masks = os.listdir(saved_mask_src)
saved_masks = [x for x in saved_masks if x.endswith('.png')]
saved_masks_full = [os.path.join(saved_mask_src,x) for x in saved_masks]
len(saved_masks)

1622

In [24]:
save_src = r"\\fatherserverdw\Kevin\hubmap\unet++\masks\blood_vessel_dilated"
for mask_src in tqdm(saved_masks_full):
    mask = Image.open(mask_src)
    maskra = np.array(mask)
    dilated_mask = binary_dilation(maskra)
    dilated_mask = dilated_mask.astype('uint8')
    name = os.path.basename(mask_src)
    Image.fromarray(dilated_mask).save(os.path.join(save_src,name))

100%|██████████| 1622/1622 [00:43<00:00, 37.00it/s]


In [3]:
new_df_train = pd.read_excel(r"\\fatherserverdw\Kevin\hubmap\unet++\bv_train_df_CV.xlsx")
new_df_train

435

100%|██████████| 435/435 [00:51<00:00,  8.45it/s]
